# Training Model: SVM

## 1. Importing libraries ...

In [1]:
import pandas as pd
import numpy as np
import setup_jwlab
from jwlab.constants import cleaned_data_filepath
from jwlab.ml_prep import prep_ml
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

## 2. Importing Participants..

In [2]:
#participants = ["904", "905", "906", "909", "910", "912", "908", "913", "914", "916", "917", "919", "920", "921", "923", "924","927", "928", "929", "930", "932"]
participants = [ "909", "910", "912", "908", "913", "914", "916", "917", "919", "920", "921", "923", "924","927", "928", "929", "930", "932"]
#9m with >40 trials
#participants = [ "909", "912", "908", "913", "914", "916", "917", "919", "920", "921", "924","927", "930"]

#12m all
#participants = ["109", "111", "112", "115", "116", "117", "119", "121", "122", "120", "124"]
#12m with >40 trials
#participants = ["109", "111", "112", "115", "124"]



## 3. Set up averaging

In [3]:
X, y, good_trial_count = prep_ml(cleaned_data_filepath, participants, downsample_num=1000, averaging="no_averaging")
Xt, yt, good_trial_count_t = prep_ml(cleaned_data_filepath, participants, downsample_num=1000, averaging="average_trials")
Xa, ya, good_trial_count_a = prep_ml(cleaned_data_filepath, participants, downsample_num=1000, averaging="average_trials_and_participants")


loaded
loaded
loaded


In [4]:
(n, d) = X[0][0].shape
assert n == y[0][0].shape[0]


In [5]:
unique, counts = np.unique(y[0][0], return_counts=True)
dict(zip(unique, counts))

{0.0: 443, 1.0: 455}

In [6]:
(n, d) = Xt[0][0].shape
assert n == yt[0][0].shape[0]


n

288

In [7]:
d

60000

In [8]:
unique, counts = np.unique(yt[0][0], return_counts=True)
dict(zip(unique, counts))

{0.0: 144, 1.0: 144}

In [9]:
(n, d) = Xa[0][0].shape
assert n == ya[0][0].shape[0]
n

16

In [10]:
d

60000

In [11]:
unique, counts = np.unique(ya[0][0], return_counts=True)
dict(zip(unique, counts))

{0.0: 8, 1.0: 8}

## 4. Setting up the SVM model ...

In [12]:
model = LinearSVC(C=1e-9, max_iter=5000)

## 5. Training and testing the model ...

### 5.1. Train on raw, test on raw (validation)

In [13]:
# from sklearn import preprocessing

# scaler = preprocessing.StandardScaler().fit(X)
# X,Xp = scaler.transform(X), scaler.transform(Xp)
model.fit(X[0][0], y[0][0])
np.mean(model.predict(X[0][0]) != y[0][0])

0.321826280623608

### 5.2. Train on raw, test on avg by trial (word repetition) 

In [14]:
# from sklearn import preprocessing

# scaler = preprocessing.StandardScaler().fit(X)
# X,Xp = scaler.transform(X), scaler.transform(Xp)
model.fit(X[0][0], y[0][0])
np.mean(model.predict(Xt[0][0]) != yt[0][0])

0.5

### 5.3. Train on raw, test on avg by word and ps 

In [15]:
# from sklearn import preprocessing

# scaler = preprocessing.StandardScaler().fit(X)
# X,Xp = scaler.transform(X), scaler.transform(Xp)
model.fit(X[0][0], y[0][0])
np.mean(model.predict(Xa[0][0]) != ya[0][0])

0.5

# 6 Subset analysis 

## 6.1 Generating random subsets of the chosen participant list

In [3]:
participants_train, participants_test = train_test_split(participants,test_size=0.2)
print(len(participants_train), len(participants_test))

14 4


### 6.2 Create train and test sets

In [4]:
X_train, y_train, good_trial_count_train = prep_ml(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="no_averaging")
X_test, y_test, good_trial_count_test = prep_ml(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="no_averaging")

#X_train, y_train, good_trial_count_train = prep_ml(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="average_trials")
#X_test, y_test, good_trial_count_test = prep_ml(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="average_trials")

#X_train, y_train, good_trial_count_train = prep_ml(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="average_trials_and_participants")
#X_test, y_test, good_trial_count_test = prep_ml(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="average_trials_and_participants")

loaded
loaded


## 6.3 Radial Classification

In [5]:
model = SVC(gamma='scale', kernel = 'rbf')

model.fit(X_train[0][0], y_train[0][0])
np.mean(model.predict(X_test[0][0]) != y_test[0][0])

0.5561797752808989

## 6.4 Alternate accuracy measurements

In [6]:
from sklearn.model_selection import train_test_split

y_pred = model.predict(X_test[0][0])

from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
print('Accuracy Score : ' + str(accuracy_score(y_test[0][0],y_pred)))
print('Precision Score : ' + str(precision_score(y_test[0][0],y_pred)))
print('Recall Score : ' + str(recall_score(y_test[0][0],y_pred)))
print('F1 Score : ' + str(f1_score(y_test[0][0],y_pred)))

from sklearn.metrics import confusion_matrix
print('Confusion Matrix : \n' + str(confusion_matrix(y_test[0][0],y_pred)))

Accuracy Score : 0.4438202247191011
Precision Score : 0.4785714285714286
Recall Score : 0.7204301075268817
F1 Score : 0.5751072961373391
Confusion Matrix : 
[[12 73]
 [26 67]]


### 7 Optimization

In [5]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
parameters = {'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000]}
svc = SVC()
model = GridSearchCV(svc, parameters, verbose=True)
model.fit(X_train[0][0], y_train[0][0])

model.cv_results_

/Users/JennMacBook/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:  1.0min finished
/Users/JennMacBook/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'mean_fit_time': array([0.71996593, 0.67748547, 0.67756327, 0.67336218, 0.67014178,
        0.67223763, 0.6778307 , 0.66928705, 0.68150608, 0.69714332,
        0.72492496, 0.70140521, 0.66463447, 0.65573414, 0.64320493,
        0.75012302, 0.70172668, 0.69788965, 0.65760986, 0.70530041]),
 'std_fit_time': array([0.06171796, 0.01710329, 0.00751038, 0.00667899, 0.01026096,
        0.01437187, 0.01219818, 0.01405444, 0.02105688, 0.04495023,
        0.0182255 , 0.01823242, 0.0232557 , 0.01171671, 0.01510925,
        0.0438018 , 0.03493424, 0.00849064, 0.02903438, 0.01077267]),
 'mean_score_time': array([0.32927004, 0.32068356, 0.32627877, 0.32848454, 0.32437229,
        0.32446202, 0.32339398, 0.32410487, 0.33187238, 0.33864633,
        0.36029434, 0.31973195, 0.29992962, 0.31019346, 0.29256916,
        0.35432355, 0.31474257, 0.30530572, 0.28757977, 0.3139174 ]),
 'std_score_time': array([0.00584223, 0.00269341, 0.00387211, 0.00445663, 0.0031799 ,
        0.00473719, 0.00470761, 0.003463

In [6]:
model.best_score_

0.55

In [7]:
model.best_estimator_

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)